In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 80
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)


class VGG(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGG, self).__init__()
        self.maxpool = nn.AdaptiveAvgPool2d(4)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1)
        #self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.avg_pool1 = nn.AvgPool2d(4)
        self.conv2 = nn.Conv2d(out_channels, out_channels=32, kernel_size=3, stride=1)
        #self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.avg_pool2 = nn.AvgPool2d(4)
        self.fc = nn.Linear(32 , 10)

    def forward(self, x):
        x = self.conv1(x)
        #print(self.conv1.size())
        #x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.avg_pool1(x)
        #print(self.conv2.size())
        #x = self.bn2(x)
        x = self.relu(x)
        x = self.avg_pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


model = VGG(3, 16).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def update_lr(optimizer, lr):
    for param_group in optimiz.param_groups:
        param_group['lr'] = lr

total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        #print(images.size())
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 ==0:
            print ("Epoch [{}/{}], step [{}/{}] Loss: {:4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the odel on the test images: {} %'.format(100*correct/total))

